In [215]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [308]:
# import data
train = pd.read_csv(r'../data/emergent/url-versions-2015-06-14-clean-train.csv', index_col=False)
test = pd.read_csv(r'../data/emergent/url-versions-2015-06-14-clean-test.csv', index_col=False)

def makeArray(text):
    text = text[1:len(text)-1]
    return np.fromstring(text,sep=' ', dtype="float")

train_Qmark_BoW = pd.read_csv(r'../datasets/claim_Qmark_BoGF_train.csv')
train_Qmark_BoW['BoGF'] = train_Qmark_BoW['BoGF'].apply(makeArray)

test_Qmark_BoW = pd.read_csv(r'../datasets/claim_Qmark_BoGF_test.csv')
test_Qmark_BoW['BoGF'] = test_Qmark_BoW['BoGF'].apply(makeArray)

train_Qmark_BoW

<ipython-input-308-e107d38d4ce3>:7: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  return np.fromstring(text,sep=' ', dtype="float")


,Unnamed: 0,claimHeadline,claimId,QFeature,BoGF
0,0,Apple will sell 19 million Apple Watches in 2015,8faeb4b0-c41b-11e4-88c9-eb158a06b9a5,0,"[0.0, 0.0, 0.0]"
1,1,Two Australian men kept a McDonald's Quarter P...,d54aaf40-b6a8-11e4-8507-b58af63d1078,0,"[0.0, 0.0, 0.0]"
2,2,Two Australian men kept a McDonald's Quarter P...,d54aaf40-b6a8-11e4-8507-b58af63d1078,0,"[0.0, 0.0, 0.0]"
3,3,Two Australian men kept a McDonald's Quarter P...,d54aaf40-b6a8-11e4-8507-b58af63d1078,0,"[0.0, 0.0, 0.0]"
4,4,Two Australian men kept a McDonald's Quarter P...,d54aaf40-b6a8-11e4-8507-b58af63d1078,0,"[0.0, 0.0, 0.0]"
...,...,...,...,...,...
2066,2066,A woman woke up during brain surgery and talke...,7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff,1,"[0.0, 0.0, 0.0]"
2067,2067,A woman woke up during brain surgery and talke...,7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff,1,"[0.0, 0.0, 0.0]"
2068,2068,A woman woke up during brain surgery and talke...,7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff,1,"[0.0, 0.0, 0.0]"
2069,2069,A woman woke up during brain surgery and talke...,7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff,1,"[0.0, 0.0, 0.0]"


In [309]:
train_Qmark_BoW = train_Qmark_BoW[["claimId", "QFeature", "BoGF"]]
test_Qmark_BoW = test_Qmark_BoW[["claimId", "QFeature", "BoGF"]]
train_Qmark_BoW.values

array([['8faeb4b0-c41b-11e4-88c9-eb158a06b9a5', 0, array([0., 0., 0.])],
       ['d54aaf40-b6a8-11e4-8507-b58af63d1078', 0, array([0., 0., 0.])],
       ['d54aaf40-b6a8-11e4-8507-b58af63d1078', 0, array([0., 0., 0.])],
       ...,
       ['7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff', 1, array([0., 0., 0.])],
       ['7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff', 1, array([0., 0., 0.])],
       ['7ca09170-93bc-11e4-b6bf-fdc8b7e3bcff', 1, array([0., 0., 0.])]],
      dtype=object)

In [310]:
# merge datasets and shuuffle with random seed
seed = 1234

train = train.merge(train_Qmark_BoW, how='inner', on="claimId")
train = sklearn.utils.shuffle(train, random_state=seed)

test = test.merge(test_Qmark_BoW, how='inner', on="claimId")
test = sklearn.utils.shuffle(test, random_state=seed)


In [311]:
train[["BoGF"]].to_numpy()

array([[array([0., 0., 0.])],
       [array([0., 0., 0.])],
       [array([0., 0., 0.])],
       ...,
       [array([2., 1., 0.])],
       [array([0., 0., 0.])],
       [array([0., 0., 0.])]], dtype=object)

In [312]:
# create train and test

X_train = train[["QFeature", "BoGF"]]
Y_train = train[["articleHeadlineStance"]]

X_test = []
Y_test = []

In [313]:
X_train

,QFeature,BoGF
15195,0,"[0.0, 0.0, 0.0]"
496,0,"[0.0, 0.0, 0.0]"
16528,0,"[0.0, 0.0, 0.0]"
23479,0,"[0.0, 0.0, 0.0]"
28898,0,"[0.0, 0.0, 0.0]"
...,...,...
27852,0,"[0.0, 0.0, 0.0]"
23605,0,"[0.0, 0.0, 0.0]"
1318,0,"[2.0, 1.0, 0.0]"
25299,0,"[0.0, 0.0, 0.0]"


In [314]:
Y_train

,articleHeadlineStance
15195,for
496,for
16528,observing
23479,for
28898,against
...,...
27852,for
23605,for
1318,observing
25299,against


In [318]:
train = train[["articleHeadlineStance", "QFeature","BoGF"]]
train_for = train[train.articleHeadlineStance == "for"]
train_against = train[train.articleHeadlineStance == "against"]
train_observing = train[train.articleHeadlineStance == "observing"]
fold_size_= int(train_for.shape[0] / 10) # fold_size per class

print("fold_size per class = \t" + str(fold_size))
print("fold_size = \t" + str(fold_size*3))

print("for size = \t" + str(train_for.shape[0]) + "\t repeat after # folds = " + str(int(train_for.shape[0] / fold_size)))
print("against size = \t" + str(train_against.shape[0]) + "\t repeat after # folds = " + str(int(train_against.shape[0] / fold_size)))
print("observing size= " + str(train_observing.shape[0]) + "\t repeat after # folds = " + str(int(train_observing.shape[0] / fold_size)))


## making stratified folds
X_folds = []
Y_folds = []

X_folds_test = []
Y_folds_test = []


print("\n")
for_i = 0
against_i = 0
observing_i = 0
for i in range(10):
    fold = pd.DataFrame()
    start = int(for_i*fold_size)
    end = int(for_i*fold_size + fold_size)
    fold = train_for[start:end]
    
    start = int(against_i*fold_size)
    end = int(against_i*fold_size + fold_size)
    if end > (train_against.shape[0]):
        against_i = 0
        start = int(against_i*fold_size)
        end = int(against_i*fold_size + fold_size)
    fold = np.append(fold, train_against[start:end], axis=0)
    
    start = int(observing_i*fold_size)
    end = int(observing_i*fold_size + fold_size)
    if end > (train_observing.shape[0]) :
        observing_i = 0
        start = int(observing_i*fold_size)
        end = int(observing_i*fold_size + fold_size)
    
    fold = np.append(fold, train_observing[start:end], axis=0)
    print("fold shape = "+ str(fold.shape))
    
    for_i = for_i + 1
    against_i = against_i + 1
    observing_i = observing_i + 1
    
    
    # add fold to X and Y
    X_folds.append(fold[:, 1:3])
    Y_folds.append(fold[:, 0])
    
X_folds = np.asarray(X_folds)
print("X_folds shape =  " + str(np.shape(X_folds)))
print("Y_folds shape =  " + str(np.shape(Y_folds)))


fold_size per class = 	1414
fold_size = 	4242
for size = 	14149	 repeat after # folds = 10
against size = 	4588	 repeat after # folds = 3
observing size= 12068	 repeat after # folds = 8


fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
fold shape = (4242, 3)
X_folds shape =  (10, 4242, 2)
Y_folds shape =  (10, 4242)


In [316]:
#create 10 stratified folds
X_strat_folds = []
Y_strat_folds = []

# considering an independent test set
X_strat_folds_test = []
Y_strat_folds_test = []

In [335]:
X_train[["BoGF"]].to_numpy()

(30805, 1)

In [336]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, max_iter = 10e3)

X = []
for i in range(len(X_train)):
    X = np.append(X, X_train)
    
soln = model.fit(X,Y_train)

print("train_acc = " + str(soln.score(X_train,Y_train))) # train accuracy
print("test_acc = " + str(soln.score(X_test,Y_test) ))# train accuracy

KeyError: 0

In [17]:
# train on folds
fold_acc = []

for i in range(10):
    model = sklearn.linear_model.LogisticRegression(penalty='l1', C=1.0, max_iter = 10e3)
    soln = model.fit(X_folds[i], Y_folds[i])
    fold_acc = np.append(fold_acc, soln.score(X_folds_test, Y_folds_test))
    
plt.plot(fold_acc);

NameError: name 'LogisticRegression' is not defined

In [ ]:
# train on stratified folds
strat_fold_acc = []

for i in range(10):
    model = sklearn.linear_model.LogisticRegression(penalty='l1', C=1.0, max_iter = 10e3)
    soln = model.fit(X_strat_folds[i], Y_strat_folds[i])
    fold_acc = np.append(fold_acc, soln.score(X_strat_folds_test, Y_strat_folds_test))
    
plt.plot(strat_fold_acc);